# Estimating error rates using HiSeq data with spiked-in EHEB clone

In [3]:
# Specifying file names
data.dir <- "C:/Users/User/projects-private/error-rate/datasets"
files <- c("S1_CONS.txt.gz", "S2_CONS.txt.gz",
           "S3_CONS.txt.gz", "S4_CONS.txt.gz")

# Reading tables and supplying them with Sample name ("$Sample" column)
load.sample <- function(file){
  
  file.name <- paste(data.dir, file, sep = "/")
  df <- read.table(gzfile(file.name), header = T, sep = "\t")
  df$Sample <- strsplit(file, "_")[[1]][1]
  df

}

# Concatenating data frames
concatenate <- function(files){
    
    dataframe <- data.frame()
    for(file in files){
        df <- load.sample(file)
        dataframe <- rbind(dataframe, df)
    }
    dataframe
    
}

dataframe <- concatenate(files)

head(dataframe)

,freq,count,v,d,j,cdr3nt,cdr3aa,mutations.FR1,mutations.CDR1,mutations.FR2,ellip.h,pol.v,pol.d.5,pol.d.3,pol.j,has.cdr3,in.frame,no.stop,complete,canonical,Sample
1,0.2983606,181334,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,NA,NA,NA,<8b>,-1,-1,-1,-1,true,true,true,true,true,S1
2,0.02974819,18080,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,NA,NA,NA,<8b>,-1,-1,-1,-1,true,true,true,true,true,S1
3,0.02297094,13961,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,NA,NA,NA,<8b>,-1,-1,-1,-1,true,true,true,true,true,S1
4,0.01999776,12154,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,NA,NA,NA,<8b>,-1,-1,-1,-1,true,true,true,true,true,S1
5,0.01752313,10650,IGHV3-74*01,IGHD2-2*01,IGHJ4*01,TGTGGAAGAGACTTCCGCGCGGGAGTCCCAGCTGAGTACTCG,CGRDFRAGVPAEYS,NA,NA,NA,<8b>,14,-1,-1,-1,true,true,true,true,false,S1
6,0.01137112,6911,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGACAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CATDYGGNAYNWFDPW,NA,NA,NA,<8b>,-1,-1,-1,-1,true,true,true,true,true,S1


In [5]:
library(dplyr)

dataframe <- as_data_frame(dataframe)

# Dropping waste
drops <- c("v.end.in.cdr3", "d.start.in.cdr3",
           "d.end.in.cdr3", "j.start.in.cdr3",
           "v.del", "d.del.5", "d.del.3", "j.del",
           "pol.v", "pol.d.5", "pol.d.3", "pol.j",
           "mutations.FR1", "mutations.CDR1",
           "mutations.FR2", "mutations.CDR2")

dataframe <- select(dataframe, -one_of(drops))

# Selecting complete CDR3 only
dataframe <- filter(dataframe, complete == "true")

# Printing part of the dataframe
print(paste("Loaded", nrow(dataframe), "clonotypes from",
            length(files), "samples."))
head(dataframe)

[1] "Loaded 460451 clonotypes from 4 samples."


,freq,count,v,d,j,cdr3nt,cdr3aa,mutations.FR3,mutations.CDR3,mutations.FR4,cdr.insert.qual,mutations.qual,has.cdr3,in.frame,no.stop,complete,canonical,Sample
1,0.2983606,181334,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,"S276:G>T,S278:G>C",,S341:C>A,HHHHHHHHHHHHHHHHHHHHHHHHH,HGH,true,true,true,true,true,S1
2,0.02974819,18080,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,"S278:G>C,S284:C>T",S327:T>C,S338:A>G,HHHH,HGHH,true,true,true,true,true,S1
3,0.02297094,13961,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,,S327:T>C,S338:A>G,HHHH,HH,true,true,true,true,true,S1
4,0.01999776,12154,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,S278:G>C,S327:T>C,S338:A>G,HHHH,GHH,true,true,true,true,true,S1
5,0.01752313,10650,IGHV3-74*01,IGHD2-2*01,IGHJ4*01,TGTGGAAGAGACTTCCGCGCGGGAGTCCCAGCTGAGTACTCG,CGRDFRAGVPAEYS,,"S289:C>G,S325:G>C",S332:A>G,HHHHHHHHHHHHHHGHHH,GGH,true,true,true,true,false,S1
6,0.01137112,6911,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,TGTGCGACAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CATDYGGNAYNWFDPW,S278:G>C,S327:T>C,"S333:G>A,S338:A>G",GHHHHH,HHGH,true,true,true,true,true,S1


In [7]:
# Dataframe containing EHEB variants
ehebs <- data.frame(variant = c("EHEB", "EHEB-V1", "EHEB-V2"),
                    seq = c("TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG",
                            "TGTGGGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG",
                            "TGTGCGACACATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG"))

# Aggregate by CDR3

dfg <- group_by(dataframe, cdr3nt, cdr3aa, v, d, j, Sample)
dfg <- summarise(dfg, freq = sum(freq), count = sum(count),
                 mutations.CDR3 = first(mutations.CDR3),
                 canonical = first(canonical),
                 no.stop = first(no.stop),
                 in.frame = first(in.frame))
dfg <- ungroup(dfg)
dfg <- arrange(dfg, desc(count), Sample)

print(nrow(dfg))
head(dfg)

[1] 441596


,cdr3nt,cdr3aa,v,d,j,Sample,freq,count,mutations.CDR3,canonical,no.stop,in.frame
1,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,S1,0.299978,182317,,true,true,true
2,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,S2,0.2027361,159948,,true,true,true
3,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,S3,0.1987603,129098,,true,true,true
4,TGTGCGAGAGATGATGGCGGGGGAAAGGGTGACTACGGAAGACTTTGG,CARDDGGGKGDYGRLW,IGHV1-18*01,IGHD4-17*01,IGHJ1*01,S4,0.1894341,121579,,true,true,true
5,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,S2,0.06360377,50180,S327:T>C,true,true,true
6,TGTGCGAGAGACTACGGTGGTAACGCCTACAACTGGTTCGACCCCTGG,CARDYGGNAYNWFDPW,IGHV4-38-2*01,IGHD4-23*01,IGHJ5*01,S1,0.07344085,44635,S327:T>C,true,true,true
